In [8]:
import pandas as pd
import pickle
import numpy as np

In [9]:
df = pickle.load(open('dataset_level2.pkl','rb'))

In [10]:
df

# batting_team
# bowling team
# city
# current score
# ball left
# wickets left
# current rr
# last five

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,2,Australia,Sri Lanka,0.1,0,0,NaN,Melbourne Cricket Ground
1,2,Australia,Sri Lanka,0.2,0,0,NaN,Melbourne Cricket Ground
2,2,Australia,Sri Lanka,0.3,1,0,NaN,Melbourne Cricket Ground
3,2,Australia,Sri Lanka,0.4,2,0,NaN,Melbourne Cricket Ground
4,2,Australia,Sri Lanka,0.5,0,0,NaN,Melbourne Cricket Ground
...,...,...,...,...,...,...,...,...
114929,961,Sri Lanka,Australia,19.3,1,0,Colombo,R Premadasa Stadium
114930,961,Sri Lanka,Australia,19.4,0,0,Colombo,R Premadasa Stadium
114931,961,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,R Premadasa Stadium
114932,961,Sri Lanka,Australia,19.6,2,0,Colombo,R Premadasa Stadium


In [12]:
df.isnull().sum()

match_id               0
batting_team           0
bowling_team           0
ball                   0
runs                   0
player_dismissed       0
city                8543
venue                  0
dtype: int64

In [13]:
df[df['city'].isnull()]['venue'].value_counts()

venue
Dubai International Cricket Stadium        2964
Pallekele International Cricket Stadium    2066
Melbourne Cricket Ground                   1453
Sydney Cricket Ground                       749
Adelaide Oval                               498
Harare Sports Club                          372
Sharjah Cricket Stadium                     249
Sylhet International Cricket Stadium        128
Carrara Oval                                 64
Name: count, dtype: int64

In [14]:
cities = np.where(df['city'].isnull(),df['venue'].str.split().apply(lambda x:x[0]),df['city'])

In [15]:
df['city'] = cities

In [16]:
df.isnull().sum()

match_id            0
batting_team        0
bowling_team        0
ball                0
runs                0
player_dismissed    0
city                0
venue               0
dtype: int64

In [17]:
df.drop(columns=['venue'],inplace=True)

In [18]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne
...,...,...,...,...,...,...,...
114929,961,Sri Lanka,Australia,19.3,1,0,Colombo
114930,961,Sri Lanka,Australia,19.4,0,0,Colombo
114931,961,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo
114932,961,Sri Lanka,Australia,19.6,2,0,Colombo


In [19]:
eligible_cities = df['city'].value_counts()[df['city'].value_counts() > 600].index.tolist()

In [20]:
df = df[df['city'].isin(eligible_cities)]

In [21]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne
...,...,...,...,...,...,...,...
114929,961,Sri Lanka,Australia,19.3,1,0,Colombo
114930,961,Sri Lanka,Australia,19.4,0,0,Colombo
114931,961,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo
114932,961,Sri Lanka,Australia,19.6,2,0,Colombo


In [30]:
# Convert 'runs' column to numeric, coercing errors to NaN
df.loc[:, 'runs'] = pd.to_numeric(df['runs'], errors='coerce')

# Calculate the cumulative sum of runs for each match
df.loc[:, 'current_score'] = df.groupby('match_id')['runs'].cumsum()

In [31]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left
0,2,Australia,Sri Lanka,0.1,0,1,Melbourne,0,0,1,1,119
1,2,Australia,Sri Lanka,0.2,0,1,Melbourne,0,0,2,2,118
2,2,Australia,Sri Lanka,0.3,1,1,Melbourne,1,0,3,3,117
3,2,Australia,Sri Lanka,0.4,2,1,Melbourne,3,0,4,4,116
4,2,Australia,Sri Lanka,0.5,0,1,Melbourne,3,0,5,5,115
...,...,...,...,...,...,...,...,...,...,...,...,...
114929,961,Sri Lanka,Australia,19.3,1,1,Colombo,125,19,3,117,3
114930,961,Sri Lanka,Australia,19.4,0,1,Colombo,125,19,4,118,2
114931,961,Sri Lanka,Australia,19.5,0,1,Colombo,125,19,5,119,1
114932,961,Sri Lanka,Australia,19.6,2,1,Colombo,127,19,6,120,0


In [33]:
df.loc[:, 'over'] = df['ball'].apply(lambda x: str(x).split(".")[0])
df.loc[:, 'ball_no'] = df['ball'].apply(lambda x: str(x).split(".")[1])

# Display the DataFrame
print(df)

        match_id batting_team bowling_team  ball  runs  player_dismissed  \
0              2    Australia    Sri Lanka   0.1     0                 1   
1              2    Australia    Sri Lanka   0.2     0                 1   
2              2    Australia    Sri Lanka   0.3     1                 1   
3              2    Australia    Sri Lanka   0.4     2                 1   
4              2    Australia    Sri Lanka   0.5     0                 1   
...          ...          ...          ...   ...   ...               ...   
114929       961    Sri Lanka    Australia  19.3     1                 1   
114930       961    Sri Lanka    Australia  19.4     0                 1   
114931       961    Sri Lanka    Australia  19.5     0                 1   
114932       961    Sri Lanka    Australia  19.6     2                 1   
114933       961    Sri Lanka    Australia  19.7     1                 1   

             city  current_score over ball_no  balls_bowled  balls_left  
0       Melbo

In [35]:
# Use .loc to modify the DataFrame directly
df.loc[:, 'balls_bowled'] = (df['over'].astype(int) * 6) + df['ball_no'].astype(int)

# Display the DataFrame
print(df)

        match_id batting_team bowling_team  ball  runs  player_dismissed  \
0              2    Australia    Sri Lanka   0.1     0                 1   
1              2    Australia    Sri Lanka   0.2     0                 1   
2              2    Australia    Sri Lanka   0.3     1                 1   
3              2    Australia    Sri Lanka   0.4     2                 1   
4              2    Australia    Sri Lanka   0.5     0                 1   
...          ...          ...          ...   ...   ...               ...   
114929       961    Sri Lanka    Australia  19.3     1                 1   
114930       961    Sri Lanka    Australia  19.4     0                 1   
114931       961    Sri Lanka    Australia  19.5     0                 1   
114932       961    Sri Lanka    Australia  19.6     2                 1   
114933       961    Sri Lanka    Australia  19.7     1                 1   

             city  current_score over ball_no  balls_bowled  balls_left  
0       Melbo

In [37]:
df.loc[:, 'balls_left'] = 120 - df['balls_bowled']
df.loc[:, 'balls_left'] = df['balls_left'].apply(lambda x: 0 if x < 0 else x)

# Display the DataFrame
print(df)

        match_id batting_team bowling_team  ball  runs  player_dismissed  \
0              2    Australia    Sri Lanka   0.1     0                 1   
1              2    Australia    Sri Lanka   0.2     0                 1   
2              2    Australia    Sri Lanka   0.3     1                 1   
3              2    Australia    Sri Lanka   0.4     2                 1   
4              2    Australia    Sri Lanka   0.5     0                 1   
...          ...          ...          ...   ...   ...               ...   
114929       961    Sri Lanka    Australia  19.3     1                 1   
114930       961    Sri Lanka    Australia  19.4     0                 1   
114931       961    Sri Lanka    Australia  19.5     0                 1   
114932       961    Sri Lanka    Australia  19.6     2                 1   
114933       961    Sri Lanka    Australia  19.7     1                 1   

             city  current_score over ball_no  balls_bowled  balls_left  
0       Melbo

In [38]:
# Convert 'player_dismissed' column values to 0 if '0', else 1
df.loc[:, 'player_dismissed'] = df['player_dismissed'].apply(lambda x: 0 if x == '0' else 1)

# Convert 'player_dismissed' column to integer type
df.loc[:, 'player_dismissed'] = df['player_dismissed'].astype('int')

# Calculate cumulative sum of 'player_dismissed' grouped by 'match_id' and modify the DataFrame directly using .loc
df.loc[:, 'player_dismissed'] = df.groupby('match_id')['player_dismissed'].cumsum()

# Calculate 'wickets_left' and modify the DataFrame directly using .loc
df.loc[:, 'wickets_left'] = 10 - df['player_dismissed']

# Display the DataFrame
print(df)


        match_id batting_team bowling_team  ball  runs  player_dismissed  \
0              2    Australia    Sri Lanka   0.1     0                 1   
1              2    Australia    Sri Lanka   0.2     0                 2   
2              2    Australia    Sri Lanka   0.3     1                 3   
3              2    Australia    Sri Lanka   0.4     2                 4   
4              2    Australia    Sri Lanka   0.5     0                 5   
...          ...          ...          ...   ...   ...               ...   
114929       961    Sri Lanka    Australia  19.3     1               122   
114930       961    Sri Lanka    Australia  19.4     0               123   
114931       961    Sri Lanka    Australia  19.5     0               124   
114932       961    Sri Lanka    Australia  19.6     2               125   
114933       961    Sri Lanka    Australia  19.7     1               126   

             city  current_score over ball_no  balls_bowled  balls_left  \
0       Melb

C:\Users\sahil\AppData\Local\Temp\ipykernel_19868\827206318.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'wickets_left'] = 10 - df['player_dismissed']


In [39]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left
0,2,Australia,Sri Lanka,0.1,0,1,Melbourne,0,0,1,1,119,9
1,2,Australia,Sri Lanka,0.2,0,2,Melbourne,0,0,2,2,118,8
2,2,Australia,Sri Lanka,0.3,1,3,Melbourne,1,0,3,3,117,7
3,2,Australia,Sri Lanka,0.4,2,4,Melbourne,3,0,4,4,116,6
4,2,Australia,Sri Lanka,0.5,0,5,Melbourne,3,0,5,5,115,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114929,961,Sri Lanka,Australia,19.3,1,122,Colombo,125,19,3,117,3,-112
114930,961,Sri Lanka,Australia,19.4,0,123,Colombo,125,19,4,118,2,-113
114931,961,Sri Lanka,Australia,19.5,0,124,Colombo,125,19,5,119,1,-114
114932,961,Sri Lanka,Australia,19.6,2,125,Colombo,127,19,6,120,0,-115


In [41]:
df.loc[:, 'crr'] = (df['current_score'] * 6) / df['balls_bowled']

# Display the DataFrame
print(df)

        match_id batting_team bowling_team  ball  runs  player_dismissed  \
0              2    Australia    Sri Lanka   0.1     0                 1   
1              2    Australia    Sri Lanka   0.2     0                 2   
2              2    Australia    Sri Lanka   0.3     1                 3   
3              2    Australia    Sri Lanka   0.4     2                 4   
4              2    Australia    Sri Lanka   0.5     0                 5   
...          ...          ...          ...   ...   ...               ...   
114929       961    Sri Lanka    Australia  19.3     1               122   
114930       961    Sri Lanka    Australia  19.4     0               123   
114931       961    Sri Lanka    Australia  19.5     0               124   
114932       961    Sri Lanka    Australia  19.6     2               125   
114933       961    Sri Lanka    Australia  19.7     1               126   

             city  current_score over ball_no  balls_bowled  balls_left  \
0       Melb

In [42]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left,crr
0,2,Australia,Sri Lanka,0.1,0,1,Melbourne,0,0,1,1,119,9,0.000000
1,2,Australia,Sri Lanka,0.2,0,2,Melbourne,0,0,2,2,118,8,0.000000
2,2,Australia,Sri Lanka,0.3,1,3,Melbourne,1,0,3,3,117,7,2.000000
3,2,Australia,Sri Lanka,0.4,2,4,Melbourne,3,0,4,4,116,6,4.500000
4,2,Australia,Sri Lanka,0.5,0,5,Melbourne,3,0,5,5,115,5,3.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114929,961,Sri Lanka,Australia,19.3,1,122,Colombo,125,19,3,117,3,-112,6.410256
114930,961,Sri Lanka,Australia,19.4,0,123,Colombo,125,19,4,118,2,-113,6.355932
114931,961,Sri Lanka,Australia,19.5,0,124,Colombo,125,19,5,119,1,-114,6.302521
114932,961,Sri Lanka,Australia,19.6,2,125,Colombo,127,19,6,120,0,-115,6.350000


In [45]:



df.loc[:, 'runs'] = pd.to_numeric(df['runs'], errors='coerce')

# Group by 'match_id'
groups = df.groupby('match_id')

# Get unique match IDs
match_ids = df['match_id'].unique()

# Initialize list to store last five runs
last_five = []

# Iterate over match IDs
for id in match_ids:
    # Apply rolling window operation to calculate sum of runs
    rolling_sum = groups.get_group(id)['runs'].rolling(window=30, min_periods=1).sum()
    
    # Extend the last_five list with the rolling sum values
    last_five.extend(rolling_sum.values.tolist())

# Display the last five runs
print(last_five)


[0.0, 0.0, 1.0, 3.0, 3.0, 6.0, 6.0, 7.0, 7.0, 7.0, 11.0, 13.0, 14.0, 15.0, 15.0, 15.0, 19.0, 19.0, 19.0, 20.0, 21.0, 22.0, 26.0, 27.0, 30.0, 33.0, 37.0, 41.0, 42.0, 43.0, 44.0, 45.0, 44.0, 42.0, 43.0, 41.0, 41.0, 41.0, 42.0, 42.0, 39.0, 38.0, 39.0, 38.0, 39.0, 40.0, 37.0, 41.0, 42.0, 42.0, 44.0, 44.0, 40.0, 39.0, 37.0, 35.0, 31.0, 27.0, 26.0, 31.0, 31.0, 30.0, 31.0, 31.0, 32.0, 31.0, 31.0, 36.0, 37.0, 37.0, 37.0, 37.0, 36.0, 36.0, 36.0, 36.0, 35.0, 32.0, 32.0, 31.0, 29.0, 29.0, 31.0, 32.0, 32.0, 32.0, 33.0, 36.0, 37.0, 37.0, 39.0, 41.0, 40.0, 41.0, 41.0, 42.0, 44.0, 39.0, 43.0, 45.0, 48.0, 48.0, 48.0, 49.0, 49.0, 49.0, 51.0, 51.0, 51.0, 52.0, 52.0, 53.0, 53.0, 53.0, 54.0, 53.0, 52.0, 50.0, 53.0, 47.0, 46.0, 45.0, 46.0, 47.0, 49.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 2.0, 8.0, 9.0, 9.0, 13.0, 14.0, 15.0, 15.0, 19.0, 20.0, 21.0, 21.0, 25.0, 26.0, 27.0, 31.0, 31.0, 31.0, 35.0, 35.0, 39.0, 40.0, 44.0, 46.0, 52.0, 53.0, 57.0, 58.0, 59.0, 59.0, 59.0, 53.0, 53.0, 54.0, 50.0, 50.0, 49.0, 50.0, 47.0,

In [47]:
df.loc[:, 'last_five'] = last_five

In [49]:
selected_columns = ['batting_team', 'bowling_team', 'city', 'current_score', 'balls_left', 'wickets_left', 'crr', 'last_five']

# Select the specified columns
df_selected = df[selected_columns]

# Display the selected DataFrame
print(df_selected)

       batting_team bowling_team       city  current_score  balls_left  \
0         Australia    Sri Lanka  Melbourne              0         119   
1         Australia    Sri Lanka  Melbourne              0         118   
2         Australia    Sri Lanka  Melbourne              1         117   
3         Australia    Sri Lanka  Melbourne              3         116   
4         Australia    Sri Lanka  Melbourne              3         115   
...             ...          ...        ...            ...         ...   
114929    Sri Lanka    Australia    Colombo            125           3   
114930    Sri Lanka    Australia    Colombo            125           2   
114931    Sri Lanka    Australia    Colombo            125           1   
114932    Sri Lanka    Australia    Colombo            127           0   
114933    Sri Lanka    Australia    Colombo            128           0   

        wickets_left       crr  last_five  
0                  9  0.000000        0.0  
1                  8  0

In [50]:
final_df = df.groupby('match_id').sum()['runs'].reset_index().merge(df,on='match_id')

In [51]:
final_df=final_df[['batting_team','bowling_team','city','current_score','balls_left','wickets_left','crr','last_five','runs_x']]

In [52]:
final_df.dropna(inplace=True)

In [53]:
final_df.isnull().sum()

batting_team     0
bowling_team     0
city             0
current_score    0
balls_left       0
wickets_left     0
crr              0
last_five        0
runs_x           0
dtype: int64

In [54]:
final_df = final_df.sample(final_df.shape[0])

In [55]:
final_df.sample(2)

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
30844,England,Pakistan,Abu Dhabi,129,0,-113,6.450000,41.0,129
19498,England,West Indies,Trinidad,116,17,-97,6.757282,24.0,121


In [56]:
X = final_df.drop(columns=['runs_x'])
y = final_df['runs_x']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [57]:
X_train

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five
33451,West Indies,Australia,Colombo,1,115,5,1.200000,1.0
15417,England,Australia,Cape Town,12,108,-2,6.000000,12.0
41349,Sri Lanka,Bangladesh,Chittagong,81,67,-43,9.169811,43.0
7165,South Africa,Sri Lanka,Johannesburg,111,40,-72,8.325000,59.0
9064,Pakistan,New Zealand,Dubai,155,7,-103,8.230088,61.0
...,...,...,...,...,...,...,...,...
23862,Australia,Pakistan,St Lucia,49,89,-21,9.483871,49.0
34270,South Africa,India,Johannesburg,37,97,-15,9.652174,37.0
6543,South Africa,Pakistan,Cape Town,148,35,-76,10.447059,68.0
48374,England,Sri Lanka,Delhi,80,50,-60,6.857143,34.0


In [63]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_absolute_error

In [65]:
trf = ColumnTransformer([
    ('trf', OneHotEncoder(drop='first'), ['batting_team', 'bowling_team', 'city'])
], remainder='passthrough')

In [66]:
pipe = Pipeline(steps=[
    ('step1',trf),
    ('step2',StandardScaler()),
    ('step3',XGBRegressor(n_estimators=1000,learning_rate=0.2,max_depth=12,random_state=1))
])

In [68]:


# Define the pipeline with OneHotEncoder and XGBRegressor
pipe = Pipeline(steps=[
    ('step1', trf),  # Assuming 'trf' is your ColumnTransformer
    ('step2', StandardScaler(with_mean=False, with_std=False)),  # Set with_mean and with_std to False for sparse matrices
    ('step3', XGBRegressor(n_estimators=1000, learning_rate=0.2, max_depth=12, random_state=1))
])

# Fit the pipeline on the training data
pipe.fit(X_train, y_train)

# Predict on the test data
y_pred = pipe.predict(X_test)

# Calculate evaluation metrics
print("R2 Score:", r2_score(y_test, y_pred))
print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))


R2 Score: 0.9313583496251157
Mean Absolute Error: 3.0936295340473383


In [71]:
pickle.dump(pipe,open('pipe.pkl','wb'))

In [70]:
import xgboost
xgboost.__version__

'2.0.3'